<a href="https://colab.research.google.com/github/mrrajatgarg/kaggle/blob/master/Training_On_Large_Dataset_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading of dataset from kaggle

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mrrajatgarg","key":"3750226d6a987e77dc47f7c58171b335"}'}

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 67 Mar 19 15:48 kaggle.json


In [3]:
!kaggle competitions download -c plant-seedlings-classification

  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 4.44MB/s]
 94% 81.0M/86.0M [00:03<00:00, 20.2MB/s]
100% 86.0M/86.0M [00:03<00:00, 29.3MB/s]
100% 1.60G/1.60G [00:36<00:00, 83.3MB/s]
100% 1.60G/1.60G [00:36<00:00, 46.7MB/s]


In [4]:
from zipfile import ZipFile

file_name = "train.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

Extracting the files...
Done


In [5]:
file_name = "test.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

Extracting the files...
Done


In [6]:
!ls

kaggle.json  sample_submission.csv.zip	test.zip  train.zip
sample_data  test			train


# Preparation of dataset - To Load the dataset in batches

In [0]:
!mkdir all_images

In [8]:
import os
import shutil

train_dir = '/content/train'
dest_dir = '/content/all_images'
counter = 0

for subdir, dirs, files in os.walk(train_dir):
    #print(files)
    for file in files:
        full_path = os.path.join(subdir, file)
        shutil.copy(full_path, dest_dir)
        counter = counter + 1
print(counter)

4750


In [9]:
import numpy as np

subdirs, dirs, files = os.walk('/content/all_images').__next__()
m = len(files)
print(m)

filenames = []
labels = np.zeros((m, 1))



import os
import shutil

images_dir = '/content/all_images'
filenames_counter = 0
labels_counter = -1

for subdir, dirs, files in os.walk(train_dir):
    #print(files)
    for file in files:
        filenames.append(file)
        labels[filenames_counter, 0] = labels_counter
        filenames_counter = filenames_counter + 1
    labels_counter = labels_counter+1
    
    
print(len(filenames))
print(labels.shape)

4750
4750
(4750, 1)


In [10]:
np.save('filenames.npy', filenames)

import keras
from keras.utils import to_categorical

y_labels_one_hot = to_categorical(labels)

np.save('y_labels_one_hot.npy', y_labels_one_hot)

Using TensorFlow backend.


# Shuffling and Splitting of dataset in Train and Validation set

In [0]:
from sklearn.utils import shuffle

filenames_shuffled, y_labels_one_hot_shuffled = shuffle(filenames, y_labels_one_hot)

np.save('y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('filenames_shuffled.npy', filenames_shuffled)

In [0]:
from sklearn.model_selection import train_test_split

In [15]:
filenames_shuffled_numpy = np.array(filenames_shuffled)

X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    filenames_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.2, random_state=1)

print(X_train_filenames.shape)
print(y_train.shape)

print(X_val_filenames.shape)
print(y_val.shape)

np.save('X_train_filenames.npy', X_train_filenames)
np.save('y_train.npy', y_train)

np.save('X_val_filenames.npy', X_val_filenames)
np.save('y_val.npy', y_val)

(3800,)
(3800, 12)
(950,)
(950, 12)


In [16]:
import shutil
shutil.make_archive("all_images", "zip", "all_images")

'/content/all_images.zip'

# Creation of Custom Generator

In [0]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return (np.array([
            resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
               for file_name in batch_x])/255), np.array(batch_y)

In [0]:
batch_size = 32

my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size)

# Defining Model Architecture and Training Model

In [0]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize


import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

In [21]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu',input_shape=(80,80,3)))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation = "relu")) #Fully connected layer
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(60, activation = "relu")) #Fully connected layer
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(12, activation = "softmax")) #Classification layer or output layer

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 76, 76, 64)        4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 76, 76, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 36, 36, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)  

In [23]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(3800 // batch_size),
                   epochs = 2,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(950 // batch_size))

Epoch 1/2


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


118/118 [==============================] - 122s 1s/step - loss: 2.0615 - acc: 0.3353 - val_loss: 8.7691 - val_acc: 0.1228
Epoch 2/2
118/118 [==============================] - 122s 1s/step - loss: 1.6545 - acc: 0.4652 - val_loss: 2.6096 - val_acc: 0.2832
